In [134]:
# Import necessary libraries
from GoogleNews import GoogleNews
import gnews
import pandas as pd
import requests
from fake_useragent import UserAgent
import newspaper
from newspaper import fulltext
import re

In [ ]:
# Define the keyword to search.
keyword = 'discpacidad'

In [136]:
# Perform news scraping from Google and extract the result into Pandas dataframe. 
googlenews = GoogleNews(lang='es', region='AR', period='1w', encode='utf-8')
googlenews.clear()
googlenews.search(keyword)
googlenews.get_page(2)
news_result = googlenews.result(sort=True)
news_data_df = pd.DataFrame.from_dict(news_result)

In [137]:
# Display information of dataframe.
news_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     20 non-null     object 
 1   media     20 non-null     object 
 2   date      20 non-null     object 
 3   datetime  0 non-null      float64
 4   desc      20 non-null     object 
 5   link      20 non-null     object 
 6   img       20 non-null     object 
dtypes: float64(1), object(6)
memory usage: 1.2+ KB


In [138]:
# Display header of dataframe.
news_data_df.head()
news_data_df['link']

0     https://salamancartvaldia.es/noticia/2025-09-1...
1     https://emisoraatlantico.com.co/judiciales/un-...
2     https://www.binoticias.com/patrulla-790/detien...
3     https://zonacero.com/judiciales/rina-dejo-un-m...
4     https://maspalomas24h.com/art/9088/la-curiosa-...
5     https://canal44.com/2025/09/14/ejecutado-en-un...
6     https://www.poresto.com/quintana-roo/sucesos/2...
7     https://www.aztecabajio.com/noticias/bomberos-...
8     https://www.radioreloj.cu/noticias-radio-reloj...
9     https://www.periodicodeibiza.es/pitiusas/ibiza...
10    https://www.poresto.com/quintana-roo/cancun/20...
11    https://noticaribe.com.mx/2025/09/14/dana-ince...
12    https://noticaribe.com.mx/2025/09/14/capturan-...
13    https://www.altagracianoticias.com/mujeres-de-...
14    https://eltiempomx.com/noticia/2025/movilizaci...
15    https://www.reynosa.gob.mx/2025/09/14/supervis...
16    https://changoonga.com/2025/09/14/apanan-en-mo...
17    https://depeso.com/riviera-maya/playa-del-

In [139]:
def clean_google_news_url(url):
    # Remove everything after the first occurrence of '.html' or trailing slash
    if '.html' in url:
        url = url.split('.html')[0] + '.html'
    elif '/' in url:
        url = url.split('&')[0]  # Remove Google tracking parameters
    return url

In [140]:
from newspaper import Article
ua = UserAgent()
news_data_df_with_text = []

for index, headers in news_data_df.iterrows():
    news_title = str(headers['title'])
    news_media = str(headers['media'])
    news_update = str(headers['date'])
    news_timestamp = str(headers['datetime'])
    news_description = str(headers['desc'])
    news_link = str(headers['link'])
    news_img = str(headers['img'])

    news_link = clean_google_news_url(news_link)

    print(news_link)

    text = ""  # initialize

    # First try Newspaper (more robust)
    try:
        article = Article(news_link, browser_user_agent=ua.chrome)
        article.download()
        article.parse()
        text = article.text
        print('Text Content via newspaper3k')
    except Exception as e:
        print(f'Newspaper extraction failed: {e}')
        # fallback to requests + fulltext
        try:
            html = requests.get(news_link, headers={'User-Agent': ua.chrome}, timeout=5).text
            text = fulltext(html)
            print('Text Content via fulltext fallback')
        except Exception as e2:
            print(f'Fallback fulltext failed: {e2}')
            # text remains empty

    news_data_df_with_text.append([
        news_title, news_media, news_update, news_timestamp,
        news_description, news_link, news_img, text
    ])

news_data_with_text_df = pd.DataFrame(
    news_data_df_with_text,
    columns=['Title', 'Media', 'Update', 'Timestamp',
             'Description', 'Link', 'Image', 'Text']
)


https://salamancartvaldia.es/noticia/2025-09-15-cantalpino-pondra-en-marcha-la-53o-vuelta-ciclista-a-salamanca-con-una-intensa-etapa-por-las-villas-376521
Text Content via newspaper3k
https://emisoraatlantico.com.co/judiciales/un-muerto-y-un-herido-deja-rina-en-villas-de-san-pablo/
Text Content via newspaper3k
https://www.binoticias.com/patrulla-790/detienen-presunto-responsable-de-robo-con-violencia-en-villas-de-nuestra-senora-de-la
Text Content via newspaper3k
https://zonacero.com/judiciales/rina-dejo-un-muerto-y-un-capturado-en-villas-de-san-pablo
Text Content via newspaper3k
https://maspalomas24h.com/art/9088/la-curiosa-obra-de-aedas-home-en-maspalomas-con-paneles-xlam
Text Content via newspaper3k
https://canal44.com/2025/09/14/ejecutado-en-un-baldio-de-villas-de-alcala-2/
Text Content via newspaper3k
https://www.poresto.com/quintana-roo/sucesos/2025/9/14/mototaxistas-se-movilizan-contra-presunto-ladron-de-parabrisas-en-villas-del-mar-3-3.html
Text Content via newspaper3k
https://w

In [141]:
# Display the entire dataframe for sample checking.
news_data_with_text_df

,Title,Media,Update,Timestamp,Description,Link,Image,Text
0,Cantalpino pondrá en marcha la 53º Vuelta Cicl...,SALAMANCArtv AL DÍA,hace 16 minutos,nan,"El próximo viernes 19 de septiembre, Cantalpin...",https://salamancartvaldia.es/noticia/2025-09-1...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","El próximo viernes 19 de septiembre, Cantalpin..."
1,Un muerto y un herido deja riña en Villas de S...,Emisora Atlántico,hace 58 minutos,nan,"En medio de un acto de intolerancia, un hombre...",https://emisoraatlantico.com.co/judiciales/un-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","En medio de un acto de intolerancia, un hombre..."
2,Detienen a presunto responsable de robo con vi...,BI Noticias,hace 4 horas,nan,Elementos de la Policía Estatal lograron la de...,https://www.binoticias.com/patrulla-790/detien...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",El hecho se registró cuando oficiales realizab...
3,Riña dejó un muerto y un capturado en Villas d...,Zona Cero,hace 4 horas,nan,Un hecho de intolerancia cobró la vida de un h...,https://zonacero.com/judiciales/rina-dejo-un-m...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Un hecho de intolerancia cobró la vida de un h...
4,La curiosa obra de AEDAS Home en Maspalomas co...,Maspalomas24h,hace 6 horas,nan,Proyecto inmobiliario en Maspalomas con villas...,https://maspalomas24h.com/art/9088/la-curiosa-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",El proyecto inmobiliario de Aedas Home en Masp...
5,Ejecutado en un baldío de Villas de Alcala,El Canal de las Noticias,hace 12 horas,nan,Comentario *. Nombre *. Correo electrónico *. ...,https://canal44.com/2025/09/14/ejecutado-en-un...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
6,Mototaxistas se movilizan contra presunto ladr...,Por Esto!,hace 12 horas,nan,Fueron alrededor de 20 las operaciones de moto...,https://www.poresto.com/quintana-roo/sucesos/2...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","La noche del domingo, alrededor de veinte oper..."
7,Bomberos controlan fuga de gas y evitan explos...,Azteca Bajío,hace 13 horas,nan,Una fuga de gas generó alarma en una vivienda ...,https://www.aztecabajio.com/noticias/bomberos-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Una fuga de gas generó alarma en una vivienda ...
8,Reinicia la Universidad Central su proceso doc...,radioreloj.cu,hace 14 horas,nan,La dirección de la Universidad Central Marta A...,https://www.radioreloj.cu/noticias-radio-reloj...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",La dirección de la Universidad Central Marta A...
9,"Una constructora de Ibiza reclama más de 1,5 m...",Periódico de Ibiza y Formentera,hace 15 horas,nan,El empresario madrileño Cristóbal Thomás de Ca...,https://www.periodicodeibiza.es/pitiusas/ibiza...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Más de un millón y medio de euros. Es la canti...


In [142]:
# Drops missiong or empty text
filtered_df = news_data_with_text_df[
    news_data_with_text_df['Text'].notna() &
    (news_data_with_text_df['Text'].str.strip() != "")
]


In [143]:
# Save the result dataframe into a CSV file.
filtered_df.to_csv("./data/news_data_with_text.csv")

In [144]:
# Reload the saved news data content from a CSV file.
news_data = pd.read_csv("./data/news_data_with_text.csv",  index_col=0)

In [164]:
from mailjet_rest import Client
import pandas as pd


# Build HTML content for the email
def build_news_html(df, max_chars=500):
    html = "<h3>Noticias de hoy</h3>"
    for _, row in df.iterrows():
        text = row['Text']
        if len(text) > max_chars:
            text = text[:max_chars].rsplit(" ", 1)[0] + "..."
        html += f"<h4>{row['Title']}</h4>"
        html += f"<h3>{row['Media']}</h3>"
        html += f"<p><strong>Descripción:</strong> {row['Description']}</p>"
        html += f"<p><strong>Texto:</strong> {text}</p>"
        html += f"<p><a href='{row['Link']}'>Leer artículo completo</a></p><hr>"
    return html


news_html = build_news_html(news_data)

# Mailjet setup
api_key = '15c19282a90f7c1e5aec38ae3653a2e3'
api_secret = 'a2db6c0dd8541c30df707c6a7350d926'

mailjet = Client(auth=(api_key, api_secret), version='v3.1')

data = {
  'Messages': [
    {
      "From": {
        "Email": "anapaulacastillozuain@gmail.com",
        "Name": "Ana"
      },
      "To": [
        {
          "Email": "anapaulacastillozuain@gmail.com",
          "Name": "Ana"
        }
      ],
      "Subject": "Noticias de hoy",
      "HTMLPart": news_html
    }
  ]
}

# Send the email
result = mailjet.send.create(data=data)
print(result.status_code)
print(result.json())

200
{'Messages': [{'Status': 'success', 'CustomID': '', 'To': [{'Email': 'anapaulacastillozuain@gmail.com', 'MessageUUID': 'eb449730-7c0a-471c-8536-2c3676168b74', 'MessageID': 576460784717386942, 'MessageHref': 'https://api.mailjet.com/v3/REST/message/576460784717386942'}], 'Cc': [], 'Bcc': []}]}
